In [3]:
import os
import csv
import pickle
import numpy as np
import pandas as pd
root = "/home/hemingzhi/.jupyter/ctr"
# table = "xtr_base"
# date = "20210527"
# table = 'qp_pv_timeseq'
# date = '20210523'
table = "xtr_v3"
date = "20210721"

# 1. 看特征的缺失情况
先用 head -n 200000 [file] > test，抽样观察。不要用完整数据

In [4]:
df = pd.read_csv(os.path.join(root, "data", table, date), sep='\t', error_bad_lines=False, quoting=csv.QUOTE_NONE, encoding='utf-8')
n = len(df)
cols = []
for fname in df.columns:
    k = len(df[fname][~df[fname].isna()])
#     if k / n >= 0.80:
    print(f"{fname:30} EXIST:{k/n}")
#         cols.append(fname)
# print('\n'.join([f"    , {c}" for c in cols]))
# print('\n'.join([f"    , SUM({c}) AS {c}" for c in cols]))

keyword                        EXIST:0.9999995341551975
item_id                        EXIST:1.0
show_cnt_7d                    EXIST:1.0
click_cnt_7d                   EXIST:1.0
play_cnt_7d                    EXIST:1.0
like_cnt_7d                    EXIST:1.0
follow_cnt_7d                  EXIST:1.0
long_view_cnt_7d               EXIST:1.0
short_view_cnt_7d              EXIST:1.0
first_click_cnt_7d             EXIST:1.0
last_click_cnt_7d              EXIST:1.0
first_view_long_cnt_7d         EXIST:1.0
last_view_long_cnt_7d          EXIST:1.0
slide_show_7d                  EXIST:1.0
slide_click_7d                 EXIST:1.0
show_cnt_30d                   EXIST:1.0
click_cnt_30d                  EXIST:1.0
play_cnt_30d                   EXIST:1.0
like_cnt_30d                   EXIST:1.0
follow_cnt_30d                 EXIST:1.0
long_view_cnt_30d              EXIST:1.0
short_view_cnt_30d             EXIST:1.0
first_click_cnt_30d            EXIST:1.0
last_click_cnt_30d             EXIST:1.0
f

In [58]:
show = "show_cnt_30d"
click = "click_cnt_30d"
dff = df[['keyword', 'item_id', show, click, 'pv']]

In [61]:
2**15

32768

In [60]:
for i in range(20):
    left, right = i*10, (i+1) * 10
    show_cond = np.logical_and(0 <= dff[show], dff[show] < right)
    show_click_cond = np.logical_and(show_cond, dff[click] <= 0)
    rate = sum(show_click_cond) / sum(show_cond)
    print(f"For the {sum(show_cond)} samples with show cnt in [{left}, {right}): rate={rate:.4f}")

For the 1258094 samples with show cnt in [0, 10): rate=0.6472
For the 2080542 samples with show cnt in [10, 20): rate=0.5111
For the 2641645 samples with show cnt in [20, 30): rate=0.4366
For the 3063803 samples with show cnt in [30, 40): rate=0.3896
For the 3398786 samples with show cnt in [40, 50): rate=0.3571
For the 3673227 samples with show cnt in [50, 60): rate=0.3333
For the 3904358 samples with show cnt in [60, 70): rate=0.3152
For the 4102697 samples with show cnt in [70, 80): rate=0.3009
For the 4275009 samples with show cnt in [80, 90): rate=0.2894
For the 4426027 samples with show cnt in [90, 100): rate=0.2799
For the 4560134 samples with show cnt in [100, 110): rate=0.2719
For the 4679297 samples with show cnt in [110, 120): rate=0.2651
For the 4786881 samples with show cnt in [120, 130): rate=0.2593
For the 4883080 samples with show cnt in [130, 140): rate=0.2542
For the 4970442 samples with show cnt in [140, 150): rate=0.2498
For the 5049860 samples with show cnt in [150

In [9]:
sum(dff[show] == 0)

0

In [93]:
ind = np.logical_and(0 < dff[show], dff[show] <= 5)
ind = np.logical_and(ind, dff['pv'] > 100)
dfff = dff[ind]
# ((dff[ind]['pv'] / 10).round() * 10).value_counts().to_dict()
# (dfff[click] / dfff[show]).round(1).value_counts().to_dict()
# (dfff[click] / dfff[show]).mean()
len(dfff['keyword'].unique())

682286

# 2. 看有无重复的键对 (query, item_id)

In [4]:
df['key'] = list(zip(df['keyword'], df['item_id']))
dup = df['key'].duplicated()
print(sum(dup))

0


# 3. 看 query 数，item_id 数

In [10]:
print(len(df))
print(len(df['keyword'].unique()) / len(df))
print(len(df['item_id'].unique()) / len(df))

21168542
0.03509325299777377
0.5809665115339545


# 4. 查看样本数

In [ ]:
n = 0
file = os.path.join(root, 'data', 'xtr_base', '20210527_filtered_train')
with open(file, 'r', encoding='utf-8') as f:
    for line in f:
        n += 1
print(n)

# 5. 生成筛选数据，令eval集的 (query, item_id) 全在train集中出现

In [ ]:
vocab = "../../data/vocab/xtr_base_20210608_filtered.pkl"
with open(vocab, 'rb') as pkl:
    sparse_feature_info = pickle.load(pkl)
    # dense_feature_info = pickle.load(pkl)
    # label_feature_info = pickle.load(pkl)
kw_idx = sparse_feature_info['keyword']['index'][0]
it_idx = sparse_feature_info['item_id']['index'][0]

In [ ]:
pre_file_root = "../../data/xtr_base/20210608_filtered_train_splits"
pre_files = sorted([os.path.join(pre_file_root, s) for s in os.listdir(pre_file_root)])

pre_keys = set()
for file in pre_files:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            feats = line.strip().split('\t')
            pre_keys.add((feats[kw_idx], feats[it_idx]))

In [ ]:
post_file_root = "../../data/xtr_base/20210609_filtered_train_splits"
post_files = sorted([os.path.join(post_file_root, s) for s in os.listdir(post_file_root)])

save_path = "../../data/xtr_base/20210609_keys_in_0608_eval_splits"
os.makedirs(save_path, exist_ok=True)


end = False
source_fp = 0
output_fp = 0
f = open(post_files[source_fp], 'r', encoding='utf-8')
while not end:
    fo = open(os.path.join(save_path, f"20210609_keys_in_0608_{output_fp}"), 'w', encoding='utf-8')
    write_cnt = 0
    while write_cnt < 200000:
        line = f.readline()
        if not line:
            if source_fp + 1 < len(post_files):
                print("DONE:", post_files[source_fp], "NOW_WRITING:", os.path.join(save_path, f"20210609_keys_in_0608_{output_fp}"), "write_cnt:", write_cnt)
                source_fp += 1
                f.close()
                f = open(post_files[source_fp], 'r', encoding='utf-8')
                continue
            else:
                end = True
                break
        try:
            feats = line.strip().split('\t')
            if (feats[kw_idx], feats[it_idx]) in pre_keys:
                fo.write(line)
                write_cnt += 1
        except:
            print(line)
    output_fp += 1
    fo.close()
f.close()

# 6. 统计eval集相比训练集缺失多少(query, item_id)

In [ ]:
vocab = "../../data/vocab/xtr_base_20210527_filtered.pkl"
with open(vocab, 'rb') as pkl:
    sparse_feature_info = pickle.load(pkl)
    # dense_feature_info = pickle.load(pkl)
    # label_feature_info = pickle.load(pkl)
kw_idx = sparse_feature_info['keyword']['index'][0]
it_idx = sparse_feature_info['item_id']['index'][0]

pre_file_root = "../../data/xtr_base/20210527_filtered_train_splits"
pre_files = sorted([os.path.join(pre_file_root, s) for s in os.listdir(pre_file_root)])

pre_keys = set()
for file in pre_files:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            feats = line.strip().split('\t')
            pre_keys.add((feats[kw_idx], feats[it_idx]))

post_file_root = "../../data/xtr_base/20210528_filtered_train_splits"
post_files = sorted([os.path.join(post_file_root, s) for s in os.listdir(post_file_root)])

post_keys = set()
for file in post_files:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            feats = line.strip().split('\t')
            post_keys.add((feats[kw_idx], feats[it_idx]))

In [ ]:
print(len(pre_keys))
print(len(post_keys))
print(len(post_keys & pre_keys))
print(len(post_keys & pre_keys) / len(post_keys))

# 7. 查看标签分布情况

In [9]:
import pandas as pd
df = pd.read_csv(os.path.join(root, "data", table, date), sep='\t')

In [5]:
ind = df['show_cnt_30d'] <= 2
dff = df[ind]
(dff['click_cnt_30d'] / dff['show_cnt_30d']).round(2).value_counts()

1.0       3476394
0.5        361536
2.0        102269
1.5         70680
inf          6993
3.0          6399
4.0          1435
2.5          1271
5.0           271
3.5           140
6.0           133
7.0            46
8.0            20
4.5            14
5.5            13
9.0            13
10.0           10
11.0            6
16.0            4
6.5             4
14.0            4
7.5             3
12.0            3
13.0            3
359.0           2
19.0            2
360.0           2
26.0            2
99.0            2
195.0           2
166.0           1
265.0           1
803.5           1
247.5           1
228.0           1
1045.0          1
227.5           1
200.0           1
168.0           1
1605.0          1
20.0            1
374.0           1
100.5           1
99.5            1
8.5             1
87.0            1
64.5            1
54.5            1
36.5            1
14.5            1
358.0           1
18.0            1
372.0           1
dtype: int64

In [13]:
(dff['long_view_cnt_30d'] / dff['play_cnt_30d']).value_counts()

0.000000    1208633
1.000000     551483
0.500000     319798
0.333333      42221
0.666667      23480
             ...   
0.409091          1
0.476190          1
0.034483          1
0.518519          1
0.809524          1
Length: 169, dtype: int64

In [9]:
dff['click_rate'] = (df['click_cnt_30d'] / df['show_cnt_30d'])
dff[df['show_cnt_30d'] == 0]['click_rate'] = 0 
dff = dff[~dff['click_rate'].isna()]
# dff['click_rate'].round(2).value_counts().to_dict()
# dff['combo'] = list(zip(dff['show_cnt'], dff['click_cnt']))
# dff['combo'].value_counts().to_dict()
ctr_mean = dff['click_rate'].mean()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [3]:
ind = np.logical_or(df['show_cnt_30d'] > 0, df['play_cnt_30d'] > 0)
dff = df[ind]
dff['lvtr'] = dff['long_view_cnt_30d'] / dff['play_cnt_30d']
dff = dff[~dff['lvtr'].isna()]
# dfff['lvtr'].round(2).value_counts().to_dict()
lvtr_mean = dff['lvtr'].mean()

In [10]:
print('ctr:', ctr_mean)
print('lvtr:', lvtr_mean)

ctr: 0.46797196359379817
lvtr: 0.3470034887590561
